# Week 2 Homework: Predicting Educational Outcomes using Logistic Regression

This week, following Chris's guest lecture, we'll be working on predicting educational outcomes using the machine learning basics we developed last week. Predicting educational outcomes is hugely important–doing so accurately would help educators to more accurately personalize learning experience and improve these outcomes. And with the rise of online learning platforms like Khan Academy and Coursera, we have the potential to apply these models at a scale never before imagined!

There are multiple ways to predict educational outcomes–perhaps the more obvious way is to use factors like previous grades and courses taken to predict how a student will do in the future. In 2016, Aulck et al. extended this idea to [predict student dropout in higher education](https://arxiv.org/abs/1606.06364). 

In this notebook, as in Chris's lecture, we will be doing more fine-grained prediction of educational outcomes. Run the below cell to start. If you are having import errors, make sure to [reinstall the requirements for this unit](http://web.stanford.edu/class/cs21si/setup.html), as they have changed slightly since last week (we are now using pandas). If you are still having setup errors, feel free to come to OH or email the class list (cs21si-staff@lists.stanford.edu). 

In [ ]:
import numpy as np
np.random.seed(42)
import pandas as pd

We are working in a setting in which students are answering questions on two concepts (think of students being quizzed on Khan Academy on algebra and calculus). Our goal is to predict how the student will perform on the next question. The model you build for this task has an immediate application: if you can predict when a student may need a bit more help, you can provide extra support through review material and hints when necessary. 

What makes this task difficult is that we will make the prediction for the next question given only the the student's answers to the last few questions, the concepts associated with these questions, and the concept for the next question! Excitingly, we'll find that we can achieve solid performance with the logistic regression model.

We will be working with a dataset Chris provided us for this assignment. Run the following cell to load the data.

In [ ]:
question_responses_df = pd.read_csv('resources/naive_c2_q50_s4000_v4.csv', header=None)
question_responses = question_responses_df.as_matrix()
print("Question responses shape: ", question_responses.shape)

index_to_concept = pd.read_csv('resources/naive_c2_q50_s4000_v4_info.txt', header=None)[2].as_matrix()
print("Index to concept shape: ", index_to_concept.shape)

*question_responses* is a NumPy array representing the correctness of answers for 4,000 students answering the same 50 questions. If question_responses[i][j] = 1, this means that the ith student answered the jth question (both 0-indexed) correctly. If it is 0, this means that the student answered this question incorrectly. Note that the questions are shared across students, so the jth question is the same for all students. 

*index_to_concept* is a NumPy array mapping the index of each question to the concept. For example, if we want to know the concept of the jth question, we evaluate index_to_concept[j]. Since the dataset we are using has questions with two concepts, this will either be a 0 or a 1. For our running example, if index_to_concept[j] = 0, then the jth question is an algebra question, and if index_to_concept[j] = 1, then the jth question is a calculus question.

For the remainder of the assignment, we will follow the same approach you might for any other real-world social good problem you encounter. The first thing an experienced machine learning practitioner does when faced with a new problem is often examine the data. Run the below cell to see some sample data for the first 5 students.

In [ ]:
question_responses_df.head()

It looks like students are usually getting questions right (there are more 1's than 0's). This is called a class imbalance, and it can make it easier to have a superficially higher accuracy–if your labels are 90% 1's, then building a dummy model that always outputs 1 will result in 90% accuracy. This is why other metrics, such as precision and recall, are often used for tasks like this. We will learn how to compute precision and recall soon. 

For now, let's quantify the class imbalance in this dataset by computing the total percent of questions that were correctly answered. This is just the mean of all of the values in *question_responses*. You will find *np.mean* useful.

In [ ]:
# YOUR CODE HERE
percent_correct = None
# END CODE
print("Percent correct: ", percent_correct)

**Expected output:**

Percent correct:  0.67229

So the class imbalance isn't as dramatic as 90%, but this tells us that a model that gets 67% accuracy might as well be guessing blindly, which establishes a baseline for us. Good to know!

After analyzing the data, the next thing an experienced machine learning practitioner might do is split it up into train, dev, and test sets. These are portions of the dataset with different functions. The train data is given to the model during training, and this is the data we use to update the model's weights and bias. The dev data is used as a means of evaluating models during training–after some number of epochs, we can see how well the model is doing on the dev set, which gives us an idea of how it is doing on data beyond the training set and helps us tune the model. The test set is a completely unseen set that is only used to evaluate the final model. We'll go over data splits again next week, so don't worry if you are unclear about the differences between train, dev, and test.

For now, split the dataset into 3 chunks as follows: the train set is the first 3000 students in the dataset, the dev set is the next 500, and the test set is the last 500 (they sum to 4000). Use NumPy indexing (which works similarly to Python indexing) to grab elements from *question_responses*. For example, if you want the first 1000 examples, you can do question_responses[0:1000].  

In [ ]:
# YOUR CODE HERE
training_question_responses = None
dev_question_responses = None
test_question_responses = None
# END CODE

print("First values in training question responses: ", training_question_responses[0][0:5])
print("First values in dev question responses: ", dev_question_responses[0][0:5])
print("First values in test question responses: ", test_question_responses[0][0:5])

**Expected output:**

First values in training question responses:  [0 1 1 1 0]

First values in dev question responses:  [1 1 1 0 0]

First values in test question responses:  [1 1 1 1 0]


Now that we have split the data, the next thing to do is think about what type of model to use. Since this is week 2, we'll be using–you guessed it–logistic regression! What might we input into our model? Remember, we want to predict how well a student will do on the next question given their previous performance and the concept associated with the next question. Ideally, we want to be able to tell our model the complete history of a student's performance and use it to make a prediction. 

However, this is not possible! To see why, imagine we are making a prediction about whether the student will correctly answer the 40th question. We will need to give the model information about the last 39 questions, along with the concept for the 40th question (algebra or calculus). If we use the same approach to make a prediction about the 41st question, we will need to give the model information about the last 40 questions, instead of 39. We can't do this, since the input into a logistic regression model is a fixed-size vector (always 300 for last week's exercises). 

The solution to this is to used a fixed window: rather than give the model all the information we can, we give it information about the last *WINDOW_SIZE* questions. Specifically, the input features (or the values that we select for our input) are: the correctness of the last *WINDOW_SIZE* questions, the concepts of the last *WINDOW_SIZE* questions, and the concept for the current question. This way, the model has enough information to make an accurate prediction, and we don't violate the fixed input constraint. Note that the fixed window involves a tradeoff: larger window sizes may result in higher accuracy, but also higher computation costs and a larger amount of data required to make a single prediction (what if a student is new to Khan Academy?). 

As a warmup, compute the size of the input required for a particular window size. This is also the size of the weights for our logistic regression model, since the dimensions of the input must match the dimensions of the weights (remember the definition of the dot product!). Note that for each of the previous WINDOW_SIZE questions, we need 2 values: 1 for the correctness (either 0 or 1), 1 for the concept (either 0 or 1, representing algebra or calculus, respectively). We also need the concept for the current question, the question we are making the prediction on.

In [ ]:
def compute_input_size(WINDOW_SIZE):
    # YOUR CODE HERE
    pass
    # END CODE
    
print("Input size for window of size 40: ", compute_input_size(40))
print("Input size for window of size 45: ", compute_input_size(45))

**Expected output:**

Input size for window of size 40:  81

Input size for window of size 45:  91

Because our data does not come in the fixed-window format described above, we need to create our dataset from *question_responses* and *index_to_concept* ourselves. Specifically, we want a function that takes in question responses for students and outputs a bunch of (input, output) pairs for our model to train and evaluate on. The input is as described above, and the output is the true correctness for the current question. 

To generate this dataset, we will iterate through students, and then iterate through windows to generate (input, output) or (x, y) pairs for each one. Note that we skip windows for which we don't have complete data, i.e. we cannot generate predictions for the first *WINDOW_SIZE* questions for each student, because they don't have enough data to feed into the window. For each complete window, the output (y) is the correctness of the student's answer to the question immediately after the window. The input is, for each question, the correctness value and the concept of the question (in that order). So the correctness for question 1 is followed by the concept of question 1, which is followed by the correctness for question 2 and then the concept of question 2. This continues for each question in the window. Finally, the last feature is the concept of the question after the window (retrieved using *index_to_concept*).

Now complete *get_student_data*, which generates (x, y) pairs from our original dataset. Ensure that your inputs follow the ordering described above exactly, since otherwise your output will not match our expected output.

In [ ]:
# For the rest of this assignment, we will be using WINDOW_SIZE = 40. 
# When you are done with the assignment, feel free to play around with this,
# but change it back before you submit!
WINDOW_SIZE = 40

# responses is a subset of our full dataset, index_to_concept is as before
def get_student_data(responses, index_to_concept):
    num_students = responses.shape[0]
    num_questions = responses.shape[1]
    data = []
    for i in range(num_students):
        # we skip questions that would have incomplete windows, this is done for you
        for j in range(WINDOW_SIZE, num_questions):
            # question j is the current question we are making predictions on
            y = responses[i][j]
            ### YOUR CODE HERE
            x = []
            # iterate over the last WINDOW_SIZE questions before question j
            for k in range(None, None): # < Change this range!!
                # Append to x correctness, then concept (using index_to_concept), in that order, 
                # for student i and question k (~2 lines)
                
                
            # Append concept of question j (~1 line)
            
            ### END CODE
            x = np.asarray(x)
            data.append((x, y))
    return data

To test this function, we can apply it to our split data as below:

In [ ]:
training_data = get_student_data(training_question_responses, index_to_concept)
dev_data = get_student_data(dev_question_responses, index_to_concept)
test_data = get_student_data(test_question_responses, index_to_concept)

print("First values for training data: ", training_data[0][0][0:15])
print("First values for dev data: ", dev_data[0][0][0:15])
print("First values for test data: ", test_data[0][0][0:15])

**Expected output:**

First values for training data:  [0 1 1 1 1 1 1 0 0 0 0 0 1 1 0]

First values for dev data:  [1 1 1 1 1 1 0 0 0 0 0 0 1 1 0]

First values for test data:  [1 1 1 1 1 1 1 0 0 0 0 0 1 1 1]

Ensure your output matches this **exactly** before proceeding. 

Now we bring back our familiar logistic model helpers. *sigmoid* and *compute_logistic_regression* are the same as before (except word vectors are no longer used in *compute_logistic_regression*), and *fit_logistic_regression* has a few changes. Go through these changes to ensure they make sense to you (they are commented). No code to write here. Try running both cells. When you run the second cell below, note that it references a function we haven't completed yet.

In [ ]:
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

def compute_logistic_regression(x, weights, bias):
    return sigmoid(np.dot(weights, x) + bias)

In [ ]:
# This function is a bit different from before! Read the comments below.
# We now pass the dev_data to training so that we can evaluate our model on it during training
def fit_logistic_regression(training_data, dev_data, NUM_EPOCHS=1000, LEARNING_RATE=0.00001):
    np.random.seed(42)
    # automatically get the size of the weights using your helper function, 
    # so we don't have to hardcode (in case we change window size)
    VEC_SIZE = compute_input_size(WINDOW_SIZE)
    weights = np.random.randn(VEC_SIZE) 
    bias = 0
    
    for epoch in range(NUM_EPOCHS):
        loss = 0
        for example in training_data:
            x, y = example
            # Note that we are no longer getting word vectors–rather, x is directly fed into the model
            pred = compute_logistic_regression(x, weights, bias)
            loss += (1 - y) * np.log(1 - pred) + y * np.log(pred)
            
            dz = pred - y
            db = dz
            # Again, we use x to compute the gradient instead of get_word_vector(x), since this is the true input
            dw = x * dz
            
            weights -= LEARNING_RATE * dw
            bias -= LEARNING_RATE * db
        if epoch % 100 == 0:
            print("\nEpoch %d, loss = %f" % (epoch, loss))      
            print("Evaluating on dev set...")
            # You will complete this function!
            evaluate_model(dev_data, weights, bias)
    return weights, bias

The above cell references *evaluate_model*, a function we haven't completed yet. We will do that now!

*evaluate_model* is a function we will use to see how our model is doing on the dev and test sets. As you can see in the above code, we will evaluate on the dev set every 100 epochs (an epoch is an iteration over every training example) during training. You will also later see that after we have fully trained our model, we will use this same function to evaluate on the test set.

*evaluate_model* computes accuracy, precision, and recall and prints out these values. As mentioned earlier, precision and recall are two metrics often used in addition to accuracy for 0-1 classification (where our prediction is between 0 and 1), especially when there is a class imbalance.

Whereas accuracy is (# correct predictions)/(# total predictions), precision is the fraction: (# correct predictions of output 1)/(# total predictions of output 1). Recall is (# correct predictions of output 1)/(# total examples with true label 1). 

Another way of writing this is that precision is (# true positives)/(# true positives + # false positives) and recall is (# true positives)/(# true positives + # false negatives). True positives are examples where the true label is 1 and the prediction is 1, false positives have true label 0 and prediction 1, true negatives have true label 0 and prediction 0, and false negatives have true label 1 and prediction 0.

Complete the *evaluate_model* function below by modifying the lines incrementing *true_positives*, *false_positives*, and *false_negatives*. Note that a prediction "is 1" when its value is > .5, and it "is 0" when its value is <= .5 (the sigmoid function never actually outputs exactly 0 or 1, since it asymptotically approaches these values). For example, for *true positives*, you want to increment if *pred* > .5 and *y* == 1. 

In [ ]:
def evaluate_model(eval_data, weights, bias):
    num_examples = len(eval_data)
    total_correct = 0.0
    true_positives = 0.0
    false_positives = 0.0
    false_negatives = 0.0
    for i in range(num_examples):
        x, y = eval_data[i]
        pred = compute_logistic_regression(x, weights, bias)
        
        total_correct += 1 if abs(pred - y) <= .5 else 0
        ### YOUR CODE HERE
        # Fill in the conditions for incrementing these quantities, 
        # using the total_correct code above as a guideline. If you are 
        # unfamiliar with Python's ternary operator (used above), look it up!
        true_positives += None
        false_positives += None
        false_negatives += None
        ### END CODE
    print("Evaluation accuracy: ", total_correct / num_examples)
    print("Precision: ", true_positives / (true_positives + false_positives))
    print("Recall: ", true_positives / (true_positives + false_negatives))

Let's see how our model does on training data with randomly initialized weights as a sanity check.

In [ ]:
np.random.seed(42)
VEC_SIZE = compute_input_size(WINDOW_SIZE)
dummy_weights = np.random.randn(VEC_SIZE) 
dummy_bias = 0
evaluate_model(training_data, dummy_weights, dummy_bias)

**Expected output:**

Evaluation accuracy:  0.23966666666666667

Precision:  0.5992578849721707

Recall:  0.06917969586635254

Our random model doesn't do well, as you'd expect.

Now that we have completed our evaluation function, we are finally ready to fit our model on the training data and see the results! Run the below cell to do so. Because we are working with a larger dataset, this will take around 10-20 minutes, depending on your computer.

In [ ]:
weights, bias = fit_logistic_regression(training_data, test_data)
print("\n\nEvaluating on unseen test data...")
evaluate_model(test_data, weights, bias)

**Last few lines of expected output (only for epoch 900 and test evaluation):**

Epoch 900, loss = -10625.872058

Evaluating on dev set...

Evaluation accuracy:  0.8468

Precision:  0.875780874579529

Recall:  0.9360554699537751


Evaluating on unseen test data...

Evaluation accuracy:  0.847

Precision:  0.8759913482335977

Recall:  0.9360554699537751

We were able to get ~0.85 accuracy on the unseen test set, which is substantially higher than a dummy model would get by predicting all 1's (0.67). Still, this is not perfect! It turns out more sophisticated models do substantially better on this task because they can capture information about previous questions beyond a fixed *WINDOW_SIZE*. We'll learn about some of these models (RNNs) and their applications to social good spaces week 9! 

Feel free to change *WINDOW_SIZE* and rerun the data generation and training cells to see what the impact is! Make sure to change it back and rerun so you get the expected output above before you submit.

Congratulations on finishing this notebook. To review, we just stepped through the entire process a machine learning practictioner might use when applying a model to a social good problem. We analyzed the data, split it up into train, dev, and test sets, came up with an approach for modeling the problem, generated our own dataset, wrote code to evaluate our progreess, and trained a model! 